<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/report_metr_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from collections import namedtuple

In [ ]:
data = pd.read_excel('/content/для анализа.xlsx', index_col='№ п/п')
data.head()

,Дата получения документа,Дата возврата документа,Номер документа,Название изделия/обозначение,"Отдел, разработчик",Количество форматов А4,Фамилия эксперта,"Проект, тема, литера/ примечания",Замечания (шифры),Повторные замечания (шифры)
№ п/п,,,,,,,,,,
1,2021-12-28,2021-12-29,ИТКС.1114-21,ИТКС.443124.001 УБР-1,"ОРС ТВГ, Малюгин",3,Докторова,NaN,NaN,NaN
2,2021-12-28,2021-12-29,ИТКС.685669.001СБ,Жгут,"ОРС ТВГ, Малюгин",4,Докторова,NaN,"10, 17",NaN
3,2021-12-28,2021-12-29,ИТКС.443124.001ИЭ,NaN,"ОРС ТВГ, Бурков",9,Докторова,NaN,"2, 19, 3, 17, 6, 7","17, 4"
4,2021-12-28,2021-12-29,ИТКС.443124.001ПЭ4,NaN,"ОРС ТВГ, Малюгин",1,Докторова,NaN,NaN,NaN
5,2021-12-28,2021-12-29,ИТКС.443124.001Е1,NaN,"ОРС ТВГ, Малюгин",2,Докторова,NaN,17,"17, 3"


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1125 entries, 1 to 1125
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Дата получения документа          1124 non-null   datetime64[ns]
 1   Дата возврата документа           1123 non-null   datetime64[ns]
 2   Номер документа                   1119 non-null   object        
 3   Название изделия/обозначение      1079 non-null   object        
 4   Отдел, разработчик                962 non-null    object        
 5   Количество форматов А4            1125 non-null   int64         
 6   Фамилия эксперта                  1123 non-null   object        
 7   Проект, тема, литера/ примечания  105 non-null    object        
 8    Замечания (шифры)                161 non-null    object        
 9   Повторные замечания (шифры)       19 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(7)
memory

# 1 Обработка пропусков в данных

In [ ]:
# заполняем пропуски в столбце 'Отдел, разработчик' значениями предыдущей строки
data['Отдел, разработчик'] = data['Отдел, разработчик'].fillna(method='ffill')

In [ ]:
data['Дата получения документа'] = data['Дата получения документа'].fillna(method='ffill')
data['Дата возврата документа'] = data['Дата возврата документа'].fillna(data['Дата получения документа'])

In [ ]:
data[' Замечания (шифры)'] = data[' Замечания (шифры)'].fillna(0)

# 2 Создание дополнительных столбцов

In [ ]:
data['Срок проверки'] = data['Дата возврата документа'] - data['Дата получения документа'] + pd.offsets.Day(1)

In [ ]:
data['Разработчик'] = ''

In [ ]:
# разделяем значения из столбца 'Отдел, разработчик' на 2 значения
for i in range(data['Отдел, разработчик'].shape[0]):
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i].split(',')
  for s in data['Отдел, разработчик'].iloc[i]:
    s = s.strip()
    data['Разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][1]
  data['Отдел, разработчик'].iloc[i] = data['Отдел, разработчик'].iloc[i][0]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# исправляем типичные ошибки в названиях отделов + актуализируем названия
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace(['ОРМД', 'ОРМС'],'ОРМ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРЛГ', 'ОРИС')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСТВГ', 'ОРС ТВГ')
data['Отдел, разработчик'] = data['Отдел, разработчик'].replace('ОРСДВ', 'ОРС ДВ')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1125 entries, 1 to 1125
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype          
---  ------                            --------------  -----          
 0   Дата получения документа          1125 non-null   datetime64[ns] 
 1   Дата возврата документа           1125 non-null   datetime64[ns] 
 2   Номер документа                   1119 non-null   object         
 3   Название изделия/обозначение      1079 non-null   object         
 4   Отдел, разработчик                1125 non-null   object         
 5   Количество форматов А4            1125 non-null   int64          
 6   Фамилия эксперта                  1123 non-null   object         
 7   Проект, тема, литера/ примечания  105 non-null    object         
 8    Замечания (шифры)                1125 non-null   object         
 9   Повторные замечания (шифры)       19 non-null     object         
 10  Срок проверки                     11

# 3 Обработка столбца с замечаниями

In [ ]:
# для удобства дальнейшей обработки создаем список шифров
for i in range(data[' Замечания (шифры)'].shape[0]):
   a = [s.strip() for s in re.split('[,.]', str(data[' Замечания (шифры)'].iloc[i]))]
   data[' Замечания (шифры)'].iloc[i] = a

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# словарь для хранения количества замечаний по каждому шифру
codes = dict(zip(np.arange(0, 31), np.zeros(30)))

In [ ]:
# функция для подсчета количества ошибок по отделам

def count_codes(data: pd.DataFrame, department: str, codes: dict) -> dict:
  # цикл по строкам датафрейма  с помощью указателя i
  
  for i in range(data.shape[0]):
    # если строка относится к нужному отделу
    if data['Отдел, разработчик'].iloc[i] == department:
      # берем значение из столбца с замечаниями
      
      for c in data[' Замечания (шифры)'].iloc[i]:
        # добавляем к счетчику шифров
        codes[int(c)] += 1
  
  return codes

In [ ]:
codes_dp = namedtuple('codes_dp', ['department', 'codes'])

counts_of_codes = [codes_dp(dp, count_codes(data, dp, codes)) for dp in data['Отдел, разработчик'].unique()]

In [ ]:
counts_of_codes[0]

codes_dp(department='ОРС ТВГ', codes={0: 979.0, 1: 8.0, 2: 29.0, 3: 9.0, 4: 37.0, 5: 41.0, 6: 16.0, 7: 13.0, 8: 3.0, 9: 10.0, 10: 20.0, 11: 4.0, 12: 28.0, 13: 0.0, 14: 0.0, 15: 7.0, 16: 1.0, 17: 110.0, 18: 12.0, 19: 43.0, 20: 2.0, 21: 3.0, 22: 1.0, 23: 0.0, 24: 2.0, 25: 1.0, 26: 1.0, 27: 25.0, 28: 2.0, 29: 5.0})

In [ ]:
data.groupby(['Отдел, разработчик']).sum()

,Количество форматов А4
"Отдел, разработчик",
КО,1463
ОГТ,334
ОН,21
ОРИС,1185
ОРМ,206
ОРНК,353
ОРС ДВ,406
ОРС ТВГ,300
ОРТА,97


In [ ]:
# разделим датасет на части поквартально и на полугодия
data_1q = data[(data['Дата получения документа'] > '2022-01-01') & (data['Дата получения документа'] < '2022-04-01')]
data_2q = data[(data['Дата получения документа'] >= '2022-04-01') & (data['Дата получения документа'] < '2022-07-01')]
data_1_half_year = data[(data['Дата получения документа'] >= '2022-01-01') & (data['Дата получения документа'] < '2022-07-01')]

In [ ]:
print(f'1 квартал: {data_1q.shape} \n2 квартал: {data_2q.shape} \nПолугодие: {data_1_half_year.shape}')

1 квартал: (493, 11) 
2 квартал: (569, 11) 
Полугодие: (1062, 11)
